# Email DEMO

In [19]:
import pandas as pd
import google.generativeai as genai
import os
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv

In [20]:
# === 1. CONFIGURATION AND SETUP ===

# --- Load Credentials Securely from .env file ---
load_dotenv()

try:
    SENDER_EMAIL = os.environ["EMAIL_SENDER"]
    SENDER_PASSWORD = os.environ["EMAIL_PASSWORD"]
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    print("Credentials loaded successfully.")
except KeyError as e:
    print(f"FATAL ERROR: Environment variable {e} not found.")
    print("Please ensure you have a .env file with EMAIL_SENDER, EMAIL_PASSWORD, and GEMINI_API_KEY.")
    exit()

Credentials loaded successfully.


In [21]:
demo_data = pd.DataFrame([
    {
        "name": "Ashelyn",
        "recipient_email": "ashelynkyne@gmail.com",
        "loyalty_profile": "Freedom Seeker",
        "tenure": 2,
        "primary_driver": "cat__Contract_Month-to-month"
    },
    {
        "name": "Michael",
        "recipient_email": "estebangr2711@gmail.com",
        "loyalty_profile": "At-Risk VIP",
        "tenure": 24,
        "primary_driver": "num__MonthlyCharges"
    },
    {
        "name": "Esteban",
        "recipient_email": "estebangr2711@gmail.com",
        "loyalty_profile": "New & Uncertain",
        "tenure": 6,
        "primary_driver": "num__tenure"
    }
])

demo_data

,name,recipient_email,loyalty_profile,tenure,primary_driver
0,Ashelyn,ashelynkyne@gmail.com,Freedom Seeker,2,cat__Contract_Month-to-month
1,Michael,estebangr2711@gmail.com,At-Risk VIP,24,num__MonthlyCharges
2,Esteban,estebangr2711@gmail.com,New & Uncertain,6,num__tenure


In [22]:
master_prompt = """
You are an expert marketing copywriter for Vodafone, a top-tier telecommunications company.
Your task is to write a personalized, persuasive, and friendly email to a valued customer who our data suggests might be at risk of leaving. The goal is to proactively address their potential concerns and retain them with a relevant offer, making them feel appreciated and understood.

### STRICT GUIDELINES TO FOLLOW ###

**1. Tone Attributes:**
   - **Friendly and Approachable:** Use warm, conversational language. Avoid technical jargon.
   - **Clear and Concise:** Messages must be straightforward and easy to digest. Use short sentences.
   - **Positive and Reassuring:** Highlight benefits and positive outcomes. Address concerns empathetically.
   - **Professional and Trustworthy:** Maintain a respectful tone. Ensure information is reliable.

**2. Email Structure:**
   - **1. Subject Line:** Must be friendly, enticing, and personalized. (e.g., "Exclusive Benefits Await You, {customer_name}!")
   - **2. Greeting:** Must be "Hi {customer_name},"
   - **3. Introduction:** Briefly and warmly explain the purpose of the email.
   - **4. Body:** Use bullet points for key information.
   - **5. Call to Action (CTA):** Must be clear and compelling. Use the given Website to add a clickable link. (e.g., "Click here to explore your exclusive benefits.")
   - **6. Closing:** Must be warm and appreciative.
   - **7. Signature:** Must be "Best regards,\nThe Vodafone Customer Care Team"

**3. Constraints:**
   - **DO NOT** mention the words "churn," "risk," "leaving," "canceling," or "unhappy." The tone must be entirely positive and proactive.
   - **DO NOT** invent specific prices or discount percentages unless they are provided in the {suggested_offer}.
   - **DO NOT** make up features or services that don't exist.

### CUSTOMER DATA & TASK ###

You will write an email for the following customer:

- **Customer Name:** {customer_name}
- **Customer Since:** {tenure_months} months
- **Loyalty Profile:** {loyalty_profile}
- **Primary Churn Driver (Internal Use Only, do not mention directly):** {primary_churn_driver}
- **Suggested Offer to Weave into the Email:** {suggested_offer}
- **Website to Insert:** https://www.vodafone.com/

Now, please write the email based on all the rules and data provided.
"""

In [23]:
# === CORE FUNCTIONS ===
def get_prompt_inputs(customer_row):
    """Translates a customer's data profile into specific inputs for the LLM prompt."""
    profile = customer_row['loyalty_profile']
    suggested_offer = ""
    if profile == "Freedom Seeker":
        suggested_offer = "A special loyalty discount on their monthly bill if they switch to a 12-month plan, offering both savings and price stability."
    elif profile == "At-Risk VIP":
        suggested_offer = "A complimentary plan review with an expert to ensure they're on the best-value tier, plus a free month of our Premium Tech Support service as a thank you."
    elif profile == "New & Uncertain":
        suggested_offer = "A welcome gift of a free 3-month trial of our Premium Online Security package to help them get the most out of their services."
    
    customer_name = customer_row['name'].split(' ')[0]
    
    return {
        "customer_name": customer_name,
        "tenure_months": customer_row['tenure'],
        "loyalty_profile": profile,  
        "primary_churn_driver": customer_row['primary_driver'],
        "suggested_offer": suggested_offer,
        "link_to_insert": "vodafone.com" 
    }

def generate_email_content(customer_data):
    try:
        model = genai.GenerativeModel('gemini-1.5-pro-latest')
        prompt_inputs = get_prompt_inputs(customer_data)
        final_prompt = master_prompt.format(**prompt_inputs)
        response = model.generate_content(final_prompt)
        return response.text
    except Exception as e:
        print(f"Error generating email content: {e}")
        return None

def parse_email_content(full_content):
    try:
        parts = full_content.split('\n\n', 1)
        subject_line = parts[0].replace("Subject: ", "").strip()
        body = parts[1].strip()
        return subject_line, body
    except Exception:
        return "A Special Offer from Vodafone", full_content

def send_email(recipient_email, subject, body):
    print(f"  > Preparing to send email to {recipient_email}...")
    msg = EmailMessage()
    msg['Subject'] = subject
    msg['From'] = SENDER_EMAIL
    msg['To'] = recipient_email
    msg.set_content(body)
    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
            smtp.send_message(msg)
        print(f" Email sent successfully to {recipient_email}!")
        return True
    except Exception as e:
        print(f" Failed to send email to {recipient_email}. Error: {e}")
        return False

# === MAIN DEMO SCRIPT ===
if __name__ == "__main__":
    print("="*60)
    print("Customer Retention - LIVE DEMO")
    print("="*60)

    if demo_data['recipient_email'].str.contains("recruiter|example.com").any():
        print("WARNING: The demo_data DataFrame contains placeholder emails.")
        print("Please edit the 'recipient_email' column before the live demo.")
        should_send_emails = False
    else:
        should_send_emails = True
        print("Recipient emails are configured. Live sending is enabled.")
        
    print("-" * 60)

    for index, customer in demo_data.iterrows():
        print(f"\n[STEP 1] Identifying customer persona: '{customer['name']}' ({customer['loyalty_profile']})")
        
        # This makes the demo flow clearer by stating the logic
        offer_logic = get_prompt_inputs(customer)['suggested_offer']
        print(f"   -> Logic: Matched profile to offer: '{offer_logic[:50]}...'")

        print("[STEP 2] Generating personalized email with Gemini AI...")
        email_content = generate_email_content(customer)
        
        if email_content:
            subject, body = parse_email_content(email_content)
            
            # Print the generated content for the audience to see
            print("\n--- GENERATED EMAIL ---")
            print(f"Subject: {subject}")            
            if should_send_emails:
                recipient = customer['recipient_email']
                print(f"[STEP 3] Sending live email to {recipient}...")
                send_email(recipient, f"DEMO: {subject}", body)
            else:
                print("[STEP 3] SKIPPED: Live email sending is disabled due to placeholder recipients.")

            print("-" * 60)
        else:
            print("Skipping email generation due to an API error.")
            print("-" * 60)
            
    print("\n✅ Live Demo Complete.")
    print("="*60)

Customer Retention - LIVE DEMO
Recipient emails are configured. Live sending is enabled.
------------------------------------------------------------

[STEP 1] Identifying customer persona: 'Ashelyn' (Freedom Seeker)
   -> Logic: Matched profile to offer: 'A special loyalty discount on their monthly bill i...'
[STEP 2] Generating personalized email with Gemini AI...

--- GENERATED EMAIL ---
Subject: Exclusive Benefits Await You, Ashelyn!
[STEP 3] Sending live email to ashelynkyne@gmail.com...
  > Preparing to send email to ashelynkyne@gmail.com...
 Email sent successfully to ashelynkyne@gmail.com!
------------------------------------------------------------

[STEP 1] Identifying customer persona: 'Michael' (At-Risk VIP)
   -> Logic: Matched profile to offer: 'A complimentary plan review with an expert to ensu...'
[STEP 2] Generating personalized email with Gemini AI...

--- GENERATED EMAIL ---
Subject: Enhancing Your Vodafone Experience, Michael!
[STEP 3] Sending live email to estebang